# RL Models Prep & Fitting

## Load data and libraries

In [4]:
# Loading packages
library(data.table)
library(nloptr)
library(ggplot2)
library(reshape2)
library(ggplot2)
library(dplyr)
library(Hmisc)
library(tidyr)
library(readr)
library(dichromat)


In [1]:
# Setting the wd
setwd("C:/Users/selin/OneDrive/Documents/GitHub/SelinProject_SVV/Jan26")
getwd()
mainDir <- getwd()

# Load the complete/cleaned task data set
PilotData <- read.csv(file = 'PilotData.csv', stringsAsFactors = TRUE)

# Number of subjects (N = 192)
nSubjects <- length(unique(PilotData$PID))
SubjectIDs <- unique(PilotData$PID)
length(SubjectIDs)

# Variable names
#names(PilotData)

[1] "C:/Users/selin/OneDrive/Documents/GitHub/SelinProject_SVV/Jan26"

[1] 192

## Prepare the data to fit the input format

In [3]:
subDir <- "ModelData"
base_par <- "/data"

for (iPar in 1:nSubjects) {
    
  # load data
  data_file <- paste0(mainDir,base_par,iPar,".csv")
  data <- read.csv(data_file)
  data <- data  %>% filter(TrialNo > 0)
  # load the relevant trial sequence
  for (i in 1:length(as.character(unique(data$Block_name)))){
      block <- as.character(unique(data$Block_name)[i])
      trial_seq_file <- paste0(mainDir, "/", block, ".csv")
      trial_seq <- read.csv(trial_seq_file)
      trial_seq <- trial_seq  %>%  select(c("Block_name", "Magnitude_stim_ac", "Magnitude_stim_bd"))
      data$Magnitude_stim_ac[which(data$Block_name == as.character(unique(data$Block_name)[i]))] <- trial_seq$Magnitude_stim_ac
      data$Magnitude_stim_bd[which(data$Block_name == as.character(unique(data$Block_name)[i]))] <- trial_seq$Magnitude_stim_bd
      }
  dir.create(file.path(mainDir, subDir))
  setwd(file.path(mainDir, subDir))
  write.csv(data, file = paste0("data_model", iPar, ".csv"))
}


In [4]:
# Merge all the individual subject data into one file

setwd("C:/Users/selin/OneDrive/Documents/GitHub/SelinProject_SVV/Jan26/ModelData")
TaskData <- plyr::ldply(list.files(), read.csv, header=TRUE)
# TaskData
names(TaskData)[1] <- "TrialNumber"

In [5]:
# Format the data set to fit the model input format
# "subject", "trialNumber", "chosenStimulus" (1, 2), "magnitudeStimulus1", "magnitudeStimulus2", "reward",
# "outcome" (0 - loss, 1 - no loss), blockSocial", "blockStable"

TaskData <- TaskData  %>% select(c(PID, TrialNumber, Optimal_stim, correct_keyboard_response1, response,
                                   Magnitude_stim_ac, Magnitude_stim_bd, MagnitudeLost, FeedbackShown, Block_name))
names(TaskData)


[1] "PID"                        "TrialNumber"               
 [3] "Optimal_stim"               "correct_keyboard_response1"
 [5] "response"                   "Magnitude_stim_ac"         
 [7] "Magnitude_stim_bd"          "MagnitudeLost"             
 [9] "FeedbackShown"              "Block_name"

In [6]:
for (i in 1:nrow(TaskData)){
    if ((TaskData$Optimal_stim[i] == "stim_ac") & (TaskData$response[i] != "None")){
        if (TaskData$correct_keyboard_response1[i] == 1){
            TaskData$chosenStimulus[i] <- 1
        } else {
            TaskData$chosenStimulus[i] <- 2
        } 
    } else if ((TaskData$Optimal_stim[i] == "stim_bd") & (TaskData$response[i] != "None")){
        if (TaskData$correct_keyboard_response1[i] == 1){
            TaskData$chosenStimulus[i] <- 2
        } else {
            TaskData$chosenStimulus[i] <- 1
        } 
    } else {
        TaskData$chosenStimulus[i] <- NA
    }
}

In [7]:
originalSubIDs <- c(as.character(unique(TaskData$PID)))
newIDs <- c(1:length(originalSubIDs))
IDs <- cbind(originalSubIDs, newIDs)
write.csv(IDs, file = paste0(mainDir,"/IDs.csv"))

In [8]:
colnames(IDs)[1] <- "PID"
TaskData <- merge(TaskData, IDs, by = "PID")
#TaskData

In [9]:
stables <- c("Block_PS1", "Block_PS2", "Block_PS3", "Block_PS4", "Block_RS1", "Block_RS2", "Block_RS3", "Block_RS4")
socials <- c("Block_PS1", "Block_PS2", "Block_PS3", "Block_PS4","Block_PV1", "Block_PV2", "Block_PV3", "Block_PV4")

TaskData <- TaskData  %>% mutate(blockSocial = ifelse(Block_name %in% socials, 1, 0))
TaskData <- TaskData  %>% mutate(blockStable = ifelse(Block_name %in% stables, 1, 0))

In [10]:
# "subject", "trialNumber", "chosenStimulus" (1, 2), "magnitudeStimulus1", "magnitudeStimulus2", "reward",
# "outcome" (0 - loss, 1 - no loss), blockSocial", "blockStable"
TaskData <- TaskData  %>% select(c(newIDs, TrialNumber, chosenStimulus, Magnitude_stim_ac, Magnitude_stim_bd, 
                                   MagnitudeLost,  FeedbackShown, blockSocial, blockStable))

In [11]:
colnames(TaskData) <- c("subject", "trialNumber", "chosenStimulus", "magnitudeStimulus1", "magnitudeStimulus2",
                        "reward", "outcome", "blockSocial", "blockStable")


In [12]:
TaskData$outcome <- as.character(TaskData$outcome)
class(TaskData$outcome)

[1] "character"

In [13]:
TaskData$outcome[which(TaskData$outcome == "Positive")] = 1
TaskData$outcome[which(TaskData$outcome == "Negative")] = 0
#TaskData

In [14]:
# Save the final data to be used as input in the models
write.csv(TaskData, file = paste0("C:/Users/selin/OneDrive/Documents/GitHub/SelinProject_SVV/","ModelInputData.csv"))